In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import mysql.connector

In [11]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="My$ql2024",
    database="restaurant_db"
)

query1 = "SELECT * FROM menu_items"
query2 = "SELECT * FROM order_details"
menu_items = pd.read_sql(query1, conn)
order_details = pd.read_sql(query2, conn)

conn.close()

d:\anaconda\envs\Deep_Learning\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
d:\anaconda\envs\Deep_Learning\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [12]:
menu_items.head()

,menu_item_id,item_name,category,price
0,101,Hamburger,American,12.95
1,102,Cheeseburger,American,13.95
2,103,Hot Dog,American,9.00
3,104,Veggie Burger,American,10.50
4,105,Mac & Cheese,American,7.00


In [13]:
order_details.head()

,order_details_id,order_id,order_date,order_time,item_id
0,1,1,2023-01-01,0 days 11:38:36,109.0
1,2,2,2023-01-01,0 days 11:57:40,108.0
2,3,2,2023-01-01,0 days 11:57:40,124.0
3,4,2,2023-01-01,0 days 11:57:40,117.0
4,5,2,2023-01-01,0 days 11:57:40,129.0
